In [ ]:
"""
Тематическое моделирование с использованием BERTopic
С визуализацией результатов
"""

# Установка необходимых библиотек
!pip3 install bertopic
!pip3 install plotly
!pip3 install wordcloud

# Импортирование библиотек
import pandas as pd
import numpy as np
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from collections import Counter
import matplotlib.pyplot as plt
import plotly.express as px
from wordcloud import WordCloud
import seaborn as sns

# Настройки визуализации
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Загрузка датасета
df = pd.read_excel('/content/События недели_19.xlsx')
docs = df['19 СОБЫТИЯ'].tolist()

# Модель для эмбеддингов
sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Настройки отображения
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

# Параметры UMAP
umap_model = UMAP(
    n_neighbors=15,
    n_components=30,
    min_dist=0.3,
    metric='cosine',
    random_state=42
)

# Параметры HDBSCAN
hdbscan_model = HDBSCAN(
    min_cluster_size=5,
    min_samples=5,
    metric='euclidean',
    prediction_data=True
)

# Инициализация модели BERTopic
topic_model = BERTopic(
    embedding_model=sentence_model,
    language="russian",
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    min_topic_size=8,
    n_gram_range=(1, 2),
    top_n_words=10,
    calculate_probabilities=True
)

# Обучение модели
topics, probs = topic_model.fit_transform(docs)

# Получение информации о темах
topic_info = topic_model.get_topic_info()
print("Информация о темах:")
print(topic_info)

# ============================================================================
# ВИЗУАЛИЗАЦИЯ 1: Распределение тем (столбчатая диаграмма)
# ============================================================================

def plot_topic_distribution(topics):
    """Визуализация распределения документов по темам."""
    topic_counts = Counter(topics)

    # Сортируем по ID темы (кроме -1, который будет первым)
    sorted_topics = sorted(topic_counts.items(), key=lambda x: (x[0] == -1, x[0]))
    topic_ids = [f"Тема {t[0]}" for t in sorted_topics]
    counts = [t[1] for t in sorted_topics]

    plt.figure(figsize=(14, 8))
    bars = plt.bar(topic_ids, counts, color='skyblue', edgecolor='navy', alpha=0.7)

    # Добавляем подписи значений
    for bar, count in zip(bars, counts):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                str(count), ha='center', va='bottom', fontsize=10)

    plt.title('Распределение документов по темам', fontsize=16, fontweight='bold')
    plt.xlabel('Темы', fontsize=12)
    plt.ylabel('Количество документов', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Выводим статистику
    total_docs = len(topics)
    noise_docs = topic_counts.get(-1, 0)
    print(f"Всего документов: {total_docs}")
    print(f"Документов в шуме (-1): {noise_docs} ({noise_docs/total_docs*100:.1f}%)")
    print(f"Уникальных тем: {len(topic_counts)}")

plot_topic_distribution(topics)

# ============================================================================
# ВИЗУАЛИЗАЦИЯ 2: Облака слов для основных тем
# ============================================================================

def plot_wordclouds(topic_model, n_topics=6):
    """Создание облаков слов для основных тем."""
    # Получаем топ-N тем по размеру (исключая шум -1)
    topic_sizes = topic_model.get_topic_info().set_index('Topic')['Count'].to_dict()

    # Исключаем тему -1 и сортируем по размеру
    valid_topics = {k: v for k, v in topic_sizes.items() if k != -1}
    top_topics = sorted(valid_topics.items(), key=lambda x: x[1], reverse=True)[:n_topics]

    # Создаем сетку графиков
    n_cols = 3
    n_rows = (n_topics + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 6*n_rows))
    if n_topics == 1:
        axes = [axes]
    else:
        axes = axes.flatten()

    for idx, (topic_id, size) in enumerate(top_topics):
        if idx >= len(axes):
            break

        # Получаем слова темы
        topic_words = topic_model.get_topic(topic_id)
        if topic_words:
            word_freq = {word: score for word, score in topic_words[:20]}

            # Создаем облако слов
            wordcloud = WordCloud(
                width=400, height=300,
                background_color='white',
                colormap='viridis',
                max_words=50,
                relative_scaling=0.5
            ).generate_from_frequencies(word_freq)

            axes[idx].imshow(wordcloud, interpolation='bilinear')
            axes[idx].set_title(f'Тема {topic_id} (размер: {size})', fontsize=14, fontweight='bold')
            axes[idx].axis('off')

    # Скрываем пустые subplots
    for idx in range(len(top_topics), len(axes)):
        axes[idx].axis('off')

    plt.tight_layout()
    plt.show()

plot_wordclouds(topic_model, n_topics=9)

# ============================================================================
# ВИЗУАЛИЗАЦИЯ 3: Топ-слова для каждой темы (горизонтальные барплоты)
# ============================================================================

def plot_topic_words(topic_model, n_topics=8, n_words=8):
    """Визуализация топ-слов для основных тем."""
    # Получаем информацию о темах
    topic_info = topic_model.get_topic_info()

    # Исключаем тему -1 и берем топ-N тем
    valid_topics = topic_info[topic_info['Topic'] != -1].head(n_topics)

    # Создаем сетку графиков
    n_cols = 2
    n_rows = (n_topics + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 6*n_rows))
    if n_topics == 1:
        axes = [axes]
    else:
        axes = axes.flatten()

    for idx, (_, topic_row) in enumerate(valid_topics.iterrows()):
        if idx >= len(axes):
            break

        topic_id = topic_row['Topic']
        topic_size = topic_row['Count']
        topic_words = topic_model.get_topic(topic_id)

        if topic_words:
            words = [word for word, score in topic_words[:n_words]]
            scores = [score for word, score in topic_words[:n_words]]

            # Инвертируем для горизонтального отображения
            y_pos = np.arange(len(words))

            axes[idx].barh(y_pos, scores, color='lightcoral', alpha=0.7)
            axes[idx].set_yticks(y_pos)
            axes[idx].set_yticklabels(words)
            axes[idx].set_xlabel('Важность')
            axes[idx].set_title(f'Тема {topic_id} (n={topic_size})', fontweight='bold')
            axes[idx].grid(axis='x', alpha=0.3)

    # Скрываем пустые subplots
    for idx in range(len(valid_topics), len(axes)):
        axes[idx].axis('off')

    plt.tight_layout()
    plt.show()

plot_topic_words(topic_model, n_topics=8)

# ============================================================================
# ВИЗУАЛИЗАЦИЯ 4: Heatmap сходства между темами
# ============================================================================

def plot_topic_similarity(topic_model):
    """Визуализация сходства между темами."""
    try:
        # Получаем матрицу сходства тем
        similarity_matrix = topic_model.visualize_topics()

        # Альтернативный способ: создаем heatmap вручную
        topic_info = topic_model.get_topic_info()
        valid_topics = topic_info[topic_info['Topic'] != -1]['Topic'].tolist()[:10]  # Берем первые 10 тем

        if len(valid_topics) > 1:
            # Создаем упрощенную матрицу сходства на основе общих слов
            similarity_data = []
            for i, topic1 in enumerate(valid_topics):
                row = []
                words1 = set([word for word, score in topic_model.get_topic(topic1)[:10]])
                for j, topic2 in enumerate(valid_topics):
                    words2 = set([word for word, score in topic_model.get_topic(topic2)[:10]])
                    common_words = len(words1.intersection(words2))
                    row.append(common_words)
                similarity_data.append(row)

            similarity_df = pd.DataFrame(
                similarity_data,
                index=[f'Тема {t}' for t in valid_topics],
                columns=[f'Тема {t}' for t in valid_topics]
            )

            plt.figure(figsize=(10, 8))
            sns.heatmap(similarity_df, annot=True, cmap='YlOrRd', fmt='d',
                       cbar_kws={'label': 'Количество общих слов'})
            plt.title('Сходство между темами (по общим словам)', fontsize=16, fontweight='bold')
            plt.tight_layout()
            plt.show()
    except Exception as e:
        print(f"Не удалось построить heatmap сходства: {e}")

plot_topic_similarity(topic_model)

# ============================================================================
# ВИЗУАЛИЗАЦИЯ 5: Временные тренды тем (если есть временные метки)
# ============================================================================

def plot_topic_timeline(df, topics):
    """Визуализация распределения тем по времени (если есть даты)."""
    # Проверяем, есть ли столбцы с датами
    date_columns = [col for col in df.columns if 'дата' in col.lower() or 'date' in col.lower()]

    if date_columns:
        print(f"Найдены столбцы с датами: {date_columns}")
        # Здесь можно добавить код для анализа временных трендов
    else:
        # Создаем искусственную временную шкалу для демонстрации
        plt.figure(figsize=(12, 6))

        # Группируем документы по темам
        topic_docs = {}
        for topic, doc in zip(topics, docs):
            if topic not in topic_docs:
                topic_docs[topic] = []
            topic_docs[topic].append(doc)

        # Берем топ-5 тем (исключая шум)
        valid_topics = [t for t in topic_docs.keys() if t != -1]
        topic_sizes = {t: len(docs) for t, docs in topic_docs.items() if t != -1}
        top_topics = sorted(topic_sizes.items(), key=lambda x: x[1], reverse=True)[:5]

        # Создаем искусственные данные для демонстрации
        time_points = np.arange(10)
        for topic_id, size in top_topics:
            # Имитируем тренд
            trend = np.random.normal(size, size*0.2, 10)
            plt.plot(time_points, trend, marker='o', linewidth=2, label=f'Тема {topic_id}')

        plt.title('Динамика популярности тем (пример)', fontsize=16, fontweight='bold')
        plt.xlabel('Временные точки')
        plt.ylabel('Количество документов')
        plt.legend()
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()

plot_topic_timeline(df, topics)

# ============================================================================
# ВИЗУАЛИЗАЦИЯ 6: Размеры кластеров в виде круговой диаграммы
# ============================================================================

def plot_topic_pie_chart(topics, top_n=10):
    """Круговая диаграмма распределения документов по темам."""
    topic_counts = Counter(topics)

    # Сортируем по размеру (исключая шум)
    sorted_counts = sorted([(k, v) for k, v in topic_counts.items() if k != -1],
                          key=lambda x: x[1], reverse=True)

    # Берем топ-N тем, остальные объединяем в "Другие"
    if len(sorted_counts) > top_n:
        main_topics = sorted_counts[:top_n]
        other_count = sum(v for k, v in sorted_counts[top_n:])
        data = main_topics + [(-2, other_count)]
        labels = [f'Тема {k}' for k, v in main_topics] + ['Другие темы']
    else:
        data = sorted_counts
        labels = [f'Тема {k}' for k, v in data]

    sizes = [v for k, v in data]

    # Добавляем тему шума отдельно
    noise_count = topic_counts.get(-1, 0)
    if noise_count > 0:
        sizes.append(noise_count)
        labels.append('Шум (-1)')

    plt.figure(figsize=(10, 8))
    colors = plt.cm.Set3(np.linspace(0, 1, len(sizes)))
    wedges, texts, autotexts = plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%',
                                      startangle=90, textprops={'fontsize': 10})

    # Улучшаем отображение процентов
    for autotext in autotexts:
        autotext.set_color('black')
        autotext.set_fontweight('bold')

    plt.title('Распределение документов по темам\n(Круговая диаграмма)',
              fontsize=16, fontweight='bold')
    plt.axis('equal')
    plt.tight_layout()
    plt.show()

plot_topic_pie_chart(topics, top_n=8)

# ============================================================================
# ВИЗУАЛИЗАЦИЯ 7: Интерактивные графики с Plotly
# ============================================================================

def plot_interactive_topics(topic_model, topics):
    """Интерактивная визуализация тем с использованием Plotly."""
    try:
        # Получаем информацию о темах для интерактивного отображения
        topic_info = topic_model.get_topic_info()

        # Создаем интерактивный барплот
        fig = px.bar(topic_info, x='Topic', y='Count',
                     title='Распределение документов по темам (интерактивное)',
                     labels={'Topic': 'ID темы', 'Count': 'Количество документов'},
                     color='Count', color_continuous_scale='viridis')

        fig.update_layout(
            xaxis=dict(tickmode='linear'),
            showlegend=False
        )
        fig.show()

        # Интерактивная круговая диаграмма для топ-10 тем
        top_topics = topic_info[topic_info['Topic'] != -1].head(10)
        fig_pie = px.pie(top_topics, values='Count', names='Topic',
                        title='Топ-10 тем по количеству документов',
                        hover_data=['Name'])
        fig_pie.show()

    except Exception as e:
        print(f"Интерактивные графики не доступны: {e}")

plot_interactive_topics(topic_model, topics)

# ============================================================================
# ДАЛЕЕ ИДЕТ ВАШ ОРИГИНАЛЬНЫЙ КОД С ОЦЕНКОЙ КАЧЕСТВА
# ============================================================================

print("\n" + "="*80)
print("ПЕРЕХОД К ОЦЕНКЕ КАЧЕСТВА МОДЕЛИ")
print("="*80)

# Создаем датафрейм для просмотра кластеров
df_after_analiz = pd.DataFrame({"Document": docs, "Topic": topics})

# Загрузка эталонных меток
df_events = pd.read_excel('07_23_События_кодировка_для_примера.xlsx', sheet_name='19 СОБЫТИЯ')

# Создаем список эталонных кодов для каждого документа
ground_truth = []
for idx, row in df_events.iterrows():
    codes = []
    if not pd.isna(row['Код1']):
        codes.append(row['Код1'])
    if not pd.isna(row['Код2']):
        codes.append(row['Код2'])
    if not pd.isna(row['Код3']):
        codes.append(row['Код3'])
    ground_truth.append(codes)

# Ручное семантическое сопоставление (один код на тему)
manual_single_code_mapping = {
    -1: 11,      # "очень_это_турция землетрясение_мобилизацию" → выбираем спецоперацию
    0: 510,      # "зо_зо зo_зо" → затрудняюсь ответить
    # ... остальной маппинг без изменений
    1: 510,     2: 250,     3: 11,      4: 11,      5: 510,
    6: 11,      7: 250,     8: 11,      9: 11,      10: 510,
    11: 11,     12: 250,    13: 510,    14: 150,    15: 11,
    16: 60,     17: 380,    18: 11,     19: 250,    20: 250,
    21: 510,    22: 250,    23: 11,     24: 510,    25: 11,
    26: 510,    27: 11,     28: 510,    29: 510,    30: 250,
    31: 510,    32: 40,     33: 250,    34: 510,    35: 510,
    36: 510,    37: 16,     38: 510,    39: 11,     40: 11,
    41: 11,     42: 40,
}

# Применяем маппинг с одним кодом
predicted_labels_single = []
for topic in topics:
    predicted_code = manual_single_code_mapping.get(topic, 510)
    predicted_labels_single.append([predicted_code])

# Функция для вычисления метрик
def calculate_metrics_single(ground_truth, predicted, code):
    tp = 0; fp = 0; fn = 0

    for i in range(len(ground_truth)):
        gt_has_code = code in ground_truth[i]
        pred_has_code = code in predicted[i]

        if gt_has_code and pred_has_code:
            tp += 1
        elif not gt_has_code and pred_has_code:
            fp += 1
        elif gt_has_code and not pred_has_code:
            fn += 1

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1, tp, fp, fn

# Вычисляем метрики для основных кодов
important_codes = [10, 11, 16, 20, 40, 60, 70, 90, 150, 210, 250, 380, 390, 510]

results_single = []
for code in important_codes:
    precision, recall, f1, tp, fp, fn = calculate_metrics_single(ground_truth, predicted_labels_single, code)
    results_single.append({
        'Код': code,
        'Precision': round(precision, 3),
        'Recall': round(recall, 3),
        'F1-Score': round(f1, 3),
        'TP': tp,
        'FP': fp,
        'FN': fn
    })

# Создаем DataFrame с результатами
results_single_df = pd.DataFrame(results_single)
print("МЕТРИКИ КАЧЕСТВА (ОДИН КОД НА ТЕМУ):")
print(results_single_df)

# Вычисляем микро-усредненные метрики
total_tp = sum(r['TP'] for r in results_single)
total_fp = sum(r['FP'] for r in results_single)
total_fn = sum(r['FN'] for r in results_single)

micro_precision_single = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
micro_recall_single = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
micro_f1_single = 2 * micro_precision_single * micro_recall_single / (micro_precision_single + micro_recall_single) if (micro_precision_single + micro_recall_single) > 0 else 0

print(f"\nМИКРО-УСРЕДНЕННЫЕ МЕТРИКИ (ОДИН КОД НА ТЕМУ):")
print(f"Micro Precision: {micro_precision_single:.3f}")
print(f"Micro Recall: {micro_recall_single:.3f}")
print(f"Micro F1-Score: {micro_f1_single:.3f}")

# Визуализация метрик качества
def plot_metrics(results_df):
    """Визуализация метрик качества по кодам."""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # Precision по кодам
    axes[0, 0].bar(results_df['Код'].astype(str), results_df['Precision'], color='lightblue', alpha=0.7)
    axes[0, 0].set_title('Precision по кодам', fontweight='bold')
    axes[0, 0].set_ylabel('Precision')
    axes[0, 0].tick_params(axis='x', rotation=45)

    # Recall по кодам
    axes[0, 1].bar(results_df['Код'].astype(str), results_df['Recall'], color='lightgreen', alpha=0.7)
    axes[0, 1].set_title('Recall по кодам', fontweight='bold')
    axes[0, 1].set_ylabel('Recall')
    axes[0, 1].tick_params(axis='x', rotation=45)

    # F1-Score по кодам
    axes[1, 0].bar(results_df['Код'].astype(str), results_df['F1-Score'], color='lightcoral', alpha=0.7)
    axes[1, 0].set_title('F1-Score по кодам', fontweight='bold')
    axes[1, 0].set_ylabel('F1-Score')
    axes[1, 0].tick_params(axis='x', rotation=45)

    # Количество TP/FP/FN для топ-5 кодов по F1
    top_codes = results_df.nlargest(5, 'F1-Score')
    x = np.arange(len(top_codes))
    width = 0.25

    axes[1, 1].bar(x - width, top_codes['TP'], width, label='TP', color='green', alpha=0.7)
    axes[1, 1].bar(x, top_codes['FP'], width, label='FP', color='red', alpha=0.7)
    axes[1, 1].bar(x + width, top_codes['FN'], width, label='FN', color='orange', alpha=0.7)

    axes[1, 1].set_title('TP/FP/FN для топ-5 кодов по F1', fontweight='bold')
    axes[1, 1].set_xticks(x)
    axes[1, 1].set_xticklabels(top_codes['Код'].astype(str))
    axes[1, 1].legend()

    plt.tight_layout()
    plt.show()

plot_metrics(results_single_df)

print(f"\n✅ Все визуализации построены успешно!")
print(f"📊 Создано 7 типов визуализаций для анализа тематического моделирования")